In [2]:
import duckdb
import pandas as pd
import pyarrow as pa

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [11]:
dbpath = "../data/bike.db" 

# create a duckdb database
con = duckdb.connect(dbpath)

In [17]:
con.close()

In [7]:
con.execute("DESCRIBE trips_view").fetchall()

[('ride_id', 'VARCHAR', 'YES', None, None, None),
 ('rideable_type', 'VARCHAR', 'YES', None, None, None),
 ('started_at', 'TIMESTAMP', 'YES', None, None, None),
 ('ended_at', 'TIMESTAMP', 'YES', None, None, None),
 ('start_station_name', 'VARCHAR', 'YES', None, None, None),
 ('start_station_id', 'VARCHAR', 'YES', None, None, None),
 ('end_station_name', 'VARCHAR', 'YES', None, None, None),
 ('end_station_id', 'VARCHAR', 'YES', None, None, None),
 ('start_lat', 'DOUBLE', 'YES', None, None, None),
 ('start_lng', 'DOUBLE', 'YES', None, None, None),
 ('end_lat', 'DOUBLE', 'YES', None, None, None),
 ('end_lng', 'DOUBLE', 'YES', None, None, None),
 ('member_casual', 'VARCHAR', 'YES', None, None, None),
 ('filename', 'VARCHAR', 'YES', None, None, None)]

In [8]:
stationId = "5788.12"

con.execute(f""" SELECT COUNT(*) FROM trips_view WHERE start_station_id = '{stationId}' """).fetchall()

[(199316,)]

In [ ]:
AllStationsNames = con.execute(f""" 
SELECT DISTINCT start_station_name
FROM trips_view
""").df()
AllStationsNames

,start_station_name
0,Broadway & W 53 St
1,W 25 St & 6 Ave
2,E 115 St & Lexington Ave
3,E 97 St & 3 Ave
4,Wilson Ave & Moffat St
...,...
2552,E New York Ave & Utica Ave
2553,92 St & 23 Ave
2554,Heights Elevator
2555,JC Medical Center


In [ ]:

AllStationsLocations = con.execute(f""" 
SELECT DISTINCT start_station_name, start_lat, start_lng
FROM trips_view
WHERE start_lat IS NOT NULL AND start_lng IS NOT NULL;
""").df()
AllStationsLocations

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,start_station_name,start_lat,start_lng
0,St Nicholas Ave & Manhattan Ave,40.809725,-73.953149
1,E 82 St & East End Ave,40.772461,-73.946821
2,E 97 St & Madison Ave,40.787801,-73.953559
3,W 49 St & 8 Ave,40.762272,-73.987882
4,Broadway & W 122 St,40.812056,-73.961255
...,...,...,...
31011736,Bayard St & Leonard St,40.719161,-73.948989
31011737,Bedford Ave & Nassau Ave,40.723174,-73.951787
31011738,Atlantic Ave & Furman St,40.691658,-74.000023
31011739,5 Ave & E 72 St,40.772839,-73.966879


In [16]:
Hotspots = con.execute(f"""
SELECT start_station_name, COUNT(*) AS nb_departures
FROM trips_view
GROUP BY start_station_name
ORDER BY nb_departures DESC
LIMIT 10;
""").df()
Hotspots

,start_station_name,nb_departures
0,W 21 St & 6 Ave,689254
1,West St & Chambers St,596355
2,University Pl & E 14 St,526948
3,1 Ave & E 68 St,525248
4,Broadway & W 25 St,515312
5,Broadway & E 14 St,514781
6,6 Ave & W 33 St,503533
7,11 Ave & W 41 St,492282
8,E 33 St & 1 Ave,485506
9,12 Ave & W 40 St,481761
